In [1]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import os
import random
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import baostock as bs
import json
bs.login()

from database_auto.db_factor_prebuilder.utils.expression_excutor import AlphaExpressionExcutor
from database_auto.db_data_downloader.downloader_base import DownloaderBase
import database_auto.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

!python --version

login success!
Python 3.8.10


In [2]:
import tensorflow as tf

def set_global_determinism(seed=1024):
    print("配置全局初始化信息...避免GPU并行差异")
    # 配置随机种子
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

    # 配置GPU并行参数
    os.environ['TF_DETERMINISTIC_OPS'] = '1' 
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

set_global_determinism()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 只显示error日志

# 只使用CPU进行训练
tf.config.set_visible_devices([], 'GPU')

# 打印Tensorflow版本
print(f"Tensorflow Version: {tf.__version__}")

# 检查是否有可用的GPU设备
if tf.test.is_built_with_cuda():
    print("TensorFlow GPU version is installed")
else:
    print("TensorFlow CPU version is installed")

# 检查TensorFlow是否能够访问GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU devices available:", gpus)
else:
    print("No GPU devices found. Running on CPU.")

# !nvidia-smi

2024-06-19 14:52:34.682552: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 14:52:35.552725: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


配置全局初始化信息...避免GPU并行差异
Tensorflow Version: 2.13.1
TensorFlow GPU version is installed
GPU devices available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# 绘图相关函数
def plot_series_dist(series):
    data = series
    plt.figure(figsize=(5,5))
    # 使用matplotlib画直方图
    plt.hist(data, bins=60, edgecolor='k', alpha=0.7)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.show()

def plot_metrics(history):
  metrics = ['loss', 'mean_squared_error']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    plt.ylim([0, plt.ylim()[1]])
    plt.legend()

def plot_cm(true_labels, pred_labels):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="g", cmap='Blues')
    plt.title('Confusion matrix')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, RobustScaler


class QuantileClipTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        
    def fit(self, X, y=None):
        # 计算给定分位数的分界值
        self.lower_bound_ = np.nanquantile(X, self.lower_quantile, axis=0)
        self.upper_bound_ = np.nanquantile(X, self.upper_quantile, axis=0)
        return self
    
    def transform(self, X, y=None):
        # 对整个数组应用剪辑操作
        return np.clip(X, self.lower_bound_, self.upper_bound_)

    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X, y).transform(X)

In [5]:
def build_alpha_factor_sync(stock_history, alpha_factor_path, exp_excutor):
    stock_history['vwap'] = stock_history[['open', 'high', 'low', 'close']].mean(axis=1)
    stock_history['returns'] = stock_history['close'].pct_change()
    alpha_factor_dict = json.loads(open(alpha_factor_path, "r").read())
    dataframe = stock_history[["code", "datetime"]]
    for alpha_name, alpha_expression in tqdm(alpha_factor_dict.items(), desc='Alpha...'):
        try:
            dataframe[alpha_name] = exp_excutor.excute(stock_history, alpha_expression)
        except Exception as e:
            dataframe[alpha_name] = np.NaN
    return dataframe

def df_to_dataset(dataframe, feature_cols, label_cols, shuffle=False, batch_size=32):
    features = dataframe[feature_cols]
    if label_cols is not None:
        labels = dataframe[label_cols]
        ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    else:
        ds = tf.data.Dataset.from_tensor_slices((dict(features)))
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(features), 10000), seed=1024)
    ds = ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [6]:
# 初始化数据库
db_conn = sqlite3.connect('../database_auto/hh_quant_auto.db')
db_downloader = DownloaderBase(db_conn, db_config)
exp_excutor = AlphaExpressionExcutor()

# 初始化基础信息
benchmark = '000016'  # 对比指数 = 上证50
train_val_test_period = {
    'train': ['2009-01-01', '2016-12-31'],
    'val': ['2017-01-01', '2018-12-31'],
    'test': ['2019-01-01', '2024-12-31']
}

train_start_date, train_end_date = train_val_test_period['train']
val_start_date, val_end_date = train_val_test_period['val']
test_start_date, test_end_date = train_val_test_period['test']

In [7]:
# 获取benchmark对应的成分股
# stock_pool = bs.query_sz50_stocks().get_data()['code'].unique()

In [8]:
# # 预计算基础数据
# stock_df_list = []
# for stock_code in tqdm(stock_pool, desc='LoadingStockData'):
#     # 1. 获取基础信息
#     stock_history = db_downloader._download_history_base_info(stock_code, train_start_date, test_end_date)
#     stock_profile = db_downloader._download_all_stock_info(stock_code)
#     stock_indicator = db_downloader._download_history_indicator_info(stock_code, train_start_date, test_end_date)
#     stock_indicator = stock_indicator.replace("", np.NaN).ffill()
#     stock_date_factor = db_downloader._download_history_date_factor_info(train_start_date, test_end_date)
#     stock_alpha_184_factor = build_alpha_factor_sync(
#         stock_history, 
#         '../database_auto/db_factor_prebuilder/factor_lib/alpha_184.json',
#         exp_excutor)
#     stock_alpha_101_factor = build_alpha_factor_sync(
#         stock_history, 
#         '../database_auto/db_factor_prebuilder/factor_lib/alpha_101.json',
#         exp_excutor)
#     stock_alpha_191_factor = build_alpha_factor_sync(
#         stock_history, 
#         '../database_auto/db_factor_prebuilder/factor_lib/alpha_191.json',
#         exp_excutor)
#     stock_df = stock_history.merge(stock_profile, on=['code']) \
#             .merge(stock_indicator, on=['code', 'datetime']) \
#             .merge(stock_date_factor, on=['datetime']) \
#             .merge(stock_alpha_184_factor, on=['code', 'datetime']) \
#             .merge(stock_alpha_101_factor, on=['code', 'datetime']) \
#             .merge(stock_alpha_191_factor, on=['code', 'datetime'])
#     stock_df_list.append(stock_df)

# # 保存原始数据
# whole_stock_df = pd.concat(stock_df_list)
# whole_stock_df.to_pickle(f'{benchmark}_{train_start_date}_{test_end_date}_base.pickle')

In [9]:
# # 预计算label数据
# def _build_reg_label(stock_dataframe, N=15):
#     stock_df = stock_dataframe.copy()
#     # 计算未来N日内的最高收益率
#     stock_df['max_return'] = stock_df['close'].rolling(window=N).max().shift(-N) / stock_df['close'] - 1
#     # 计算未来N日内的最低收益率
#     stock_df['min_return'] = stock_df['close'].rolling(window=N).min().shift(-N) / stock_df['close'] - 1
#     # 计算未来N日内的收益率和（期望最高收益率越高越好，最低收益率也越高越好，由于最低收益率是负数，因此使用最高+最低来作为综合收益指标）
#     stock_df['label'] = stock_df['max_return'] + stock_df['min_return']
#     # 过滤第二天一字涨停股票
#     stock_df = stock_df[stock_df['high'].shift(-1) != stock_df['low'].shift(-1)]
#     return stock_df[['code', 'datetime', 'label']]

# stock_label_list = []
# for stock_code in tqdm(stock_pool, desc='LoadingStockData'):
#     # 1. 获取基础信息
#     stock_history = db_downloader._download_history_base_info(stock_code, train_start_date, test_end_date)
#     stock_label = _build_reg_label(stock_history)
#     stock_label_list.append(stock_label)

# # 保存标签数据
# whole_label_df = pd.concat(stock_label_list)
# whole_label_df.to_pickle(f'{benchmark}_{train_start_date}_{test_end_date}_label.pickle')

In [7]:
factor_alpha_184_columns = [f"alpha_184_{i+1}" for i in range(184)]
feature_config = {
    "numeric_features": ['turnover_rate', 'pe_ttm', 'ps_ttm', 'pcf_ncf_ttm', 'pb_mrq'] + factor_alpha_184_columns,
    "integer_categorical_features": ['month'],
    "string_categorical_features": ['industry', 'season'],
}
full_feature_columns = ['code', 'code_name', 'datetime'] + [i for feas in feature_config.values() for i in feas]

In [8]:
# 进行基本的数据清理
def base_dataframe_cleaner(dataframe):
    stock_df = dataframe.copy()
    stock_df = stock_df.replace("", np.NaN).ffill()
    stock_df = stock_df.replace([np.inf, -np.inf], np.nan)
    stock_df = stock_df.dropna(axis=1, how='all') # 过滤特征列全为空的column
    stock_df = stock_df.dropna() # 剔除包含空值的行
    return stock_df

In [9]:
# 从本地加载Base数据
stock_base_df_raw = pd.read_pickle('./000016_2006-01-01_2024-12-31_base.pickle')
stock_base_df = base_dataframe_cleaner(stock_base_df_raw[full_feature_columns])
# 从本地加载Label数据
stock_label_df = pd.read_pickle('./000016_2006-01-01_2024-12-31_label.pickle')
stock_merge_df = stock_base_df.merge(stock_label_df, on=['code', 'datetime'])
stock_merge_df.shape

(170869, 196)

In [ ]:
# 预计算label数据，考虑时间衰减
def _build_reg_label(stock_dataframe, N=15):
    stock_df = stock_dataframe.copy()
    # 计算未来N日内的最高收益率
    stock_df['max_return'] = stock_df['close'].rolling(window=N).max().shift(-N) / stock_df['close'] - 1
    # 计算未来N日内的最低收益率
    stock_df['min_return'] = stock_df['close'].rolling(window=N).min().shift(-N) / stock_df['close'] - 1
    # 计算未来N日内的收益率和（期望最高收益率越高越好，最低收益率也越高越好，由于最低收益率是负数，因此使用最高+最低来作为综合收益指标）
    stock_df['label'] = stock_df['max_return'] + stock_df['min_return']
    # 过滤第二天一字涨停股票
    stock_df = stock_df[stock_df['high'].shift(-1) != stock_df['low'].shift(-1)]
    return stock_df[['code', 'datetime', 'label']]

stock_label_list = []
for stock_code in tqdm(stock_pool, desc='LoadingStockData'):
    # 1. 获取基础信息
    stock_history = db_downloader._download_history_base_info(stock_code, train_start_date, test_end_date)
    stock_label = _build_reg_label(stock_history)
    stock_label_list.append(stock_label)

# 保存标签数据
whole_label_df = pd.concat(stock_label_list)
whole_label_df.to_pickle(f'{benchmark}_{train_start_date}_{test_end_date}_label.pickle')

In [66]:

# for i in range(len(group)):
#         price1 = group.at[group.index[i], 'close']
#         atr = group.at[group.index[i], 'ATR']
#         low = price1 - rate * atr
#         high = price1 + rate * atr
#         future_prices = group['close'].iloc[i+1:i+15] # 获取当前日期之后14天的价格
#         group.at[group.index[i], 'high_loss'] = high
#         group.at[group.index[i], 'low_loss'] = low
#         if len(future_prices) > 0:  # 确保14天内有数据
#             if (future_prices < low).any():  # 如果14天内存在价格小于止损价
#                 a = future_prices[future_prices < low].index[0]  # 找到第一个价格小于止损价的日期
#                 if (group['close'].loc[group.index[i]:a-1] < high).all():  # 检查之前到当天的价格是否都小于最小止盈价
#                     group.at[group.index[i], 'downtrend'] = 1  # 满足条件时设置为1


import pandas as pd

# 创建一个包含数字的Series
data = pd.Series([3, 7, 5, 2, 8, 4, 6, 1])

data1 = data.iloc[3:8]
# 给定值 x
x = 5

# 使用条件筛选和idxmax来找到第一个小于x的值的索引
index = data[data < x].index[1]

print(index)

3


In [32]:
import numpy as np

# # 假设 x 是包含 float 类型的 14 个元素的数组
# x1 = np.random.uniform(-0.1, 0.1, 14)
# x2 = np.random.uniform(-0.1, 0.1, 14)

# # 计算 y
# def func(x):
#     # return np.sum((x + 0.) / (np.arange(1, 15)**0.4/0.5))
#     return np.sum((x + 0.) / (np.arange(1, 15)*0.4+0.3))
# print("x1 值为:", x1)
# print("x2 值为:", x2)
# # 打印结果
# print("计算得到的 y1 值为:", func(x1))
# print("计算得到的 y2 值为:", func(x2))

print(5/(np.arange(1,15)+4)+0.75)
print(0.85**np.arange(1,15)+0.9)
print(-np.log(np.arange(1,15)+1)*0.36+1)

[1.75       1.58333333 1.46428571 1.375      1.30555556 1.25
 1.20454545 1.16666667 1.13461538 1.10714286 1.08333333 1.0625
 1.04411765 1.02777778]
[1.75       1.6225     1.514125   1.42200625 1.34370531 1.27714952
 1.22057709 1.17249053 1.13161695 1.0968744  1.06734324 1.04224176
 1.02090549 1.00276967]
[0.75046701 0.60449958 0.50093403 0.42060235 0.35496659 0.29947235
 0.25140104 0.20899915 0.17106937 0.1367577  0.10543361 0.07661823
 0.04993936 0.02510193]


In [13]:
print("开始拆分训练、验证、测试集合...")
def split_data_by_date(dataframe, start_date, end_date):
    return dataframe[(dataframe['datetime'] >= start_date) & (dataframe['datetime'] <= end_date)]

train_data = split_data_by_date(stock_merge_df, train_start_date, train_end_date)
val_data = split_data_by_date(stock_merge_df, val_start_date, val_end_date)
test_data = split_data_by_date(stock_base_df, test_start_date, test_end_date)

print(f"train_data_size: {train_data.shape}")
print(f"val_data_size: {val_data.shape}")
print(f"test_data_size: {test_data.shape}")

开始拆分训练、验证、测试集合...
train_data_size: (71297, 196)
val_data_size: (19998, 196)
test_data_size: (61842, 195)


In [14]:
print("开始对特征进行预处理...")
feature_preprocess_pipeline = Pipeline(steps=[
    ('robust_scaler', RobustScaler()),
    ('minmax_scaler', MinMaxScaler()),
])
preprocess_feature_columns = feature_config.get('numeric_features', [])
train_data[preprocess_feature_columns] = feature_preprocess_pipeline.fit_transform(train_data[preprocess_feature_columns])
val_data[preprocess_feature_columns] = feature_preprocess_pipeline.transform(val_data[preprocess_feature_columns])
test_data[preprocess_feature_columns] = feature_preprocess_pipeline.transform(test_data[preprocess_feature_columns])

开始对特征进行预处理...


In [15]:
print("开始对标签进行预处理...")
label_preprocess_pipeline = Pipeline(steps=[
    ('quantile_clipper', QuantileClipTransformer()),
    ('robust_scaler', RobustScaler()),
    ('minmax_scaler', MinMaxScaler()),
])
preprocess_target_columns = ['label']
train_data[preprocess_target_columns] = label_preprocess_pipeline.fit_transform(train_data[preprocess_target_columns])
val_data[preprocess_target_columns] = label_preprocess_pipeline.transform(val_data[preprocess_target_columns])

开始对标签进行预处理...


In [16]:
# 转换为tensorflow所使用的dataset
print("开始将DataFrame转换为DataSet...")
batch_size = 1024
feature_columns = feature_config.get('numeric_features', []) + feature_config.get('integer_categorical_features', []) + feature_config.get('string_categorical_features', [])
label_columns = ['label']
# 将data转换为dataset
train_ds = df_to_dataset(train_data, feature_columns, label_columns, shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(val_data, feature_columns, label_columns, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test_data, feature_columns, None, shuffle=False, batch_size=batch_size)  

开始将DataFrame转换为DataSet...


In [17]:
# 准备模型训练
print("开始模型初始化 & 训练...")
from models.single_task.model_moe import QuantModel
model_config = {
        "seed": 1024,
        "feature_use_embedding": False,
        "feature_embedding_dims": 4,
        "numeric_features_with_boundaries": {k: pd.qcut(train_data[k], q=20, retbins=True, duplicates='drop')[1].tolist() for k in feature_config.get('numeric_features', [])},
        "integer_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('integer_categorical_features', [])},
        "string_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('string_categorical_features', [])},
    }
model = QuantModel(config=model_config)

initial_learning_rate = 5e-4

model.compile(
    optimizer=tf.keras.optimizers.Adam(initial_learning_rate),
    loss=tf.keras.losses.Huber(delta=0.15),
    )

EPOCHS = 500

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

baseline_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks = [early_stopping],
)

开始模型初始化 & 训练...


Epoch 1/500


2024-06-13 02:16:14.504359: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x564cae6d6700 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-06-13 02:16:14.504427: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-06-13 02:16:14.508904: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-13 02:16:14.535803: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


70/70 [==============================] - 18s 165ms/step - loss: 0.1086 - val_loss: 0.0322
Epoch 2/500
70/70 [==============================] - 5s 73ms/step - loss: 0.0670 - val_loss: 0.0266
Epoch 3/500
70/70 [==============================] - 5s 68ms/step - loss: 0.0518 - val_loss: 0.0223
Epoch 4/500
70/70 [==============================] - 5s 72ms/step - loss: 0.0398 - val_loss: 0.0171
Epoch 5/500
70/70 [==============================] - 5s 72ms/step - loss: 0.0315 - val_loss: 0.0149
Epoch 6/500
70/70 [==============================] - 5s 70ms/step - loss: 0.0250 - val_loss: 0.0121
Epoch 7/500
70/70 [==============================] - 5s 69ms/step - loss: 0.0210 - val_loss: 0.0113
Epoch 8/500
70/70 [==============================] - 5s 72ms/step - loss: 0.0186 - val_loss: 0.0109
Epoch 9/500
70/70 [==============================] - 5s 74ms/step - loss: 0.0174 - val_loss: 0.0106
Epoch 10/500
70/70 [==============================] - 5s 71ms/step - loss: 0.0162 - val_loss: 0.0104
Epoch 11/

In [18]:
print("开始保存回测预测结果...")
model_pred_result = model.predict(test_ds)
output_df = test_data[['code', 'code_name', 'datetime']]
output_df['label_pred'] = tf.sigmoid(model_pred_result)
output_df = output_df.rename(columns={
    'code': 'stock_code',
    'code_name': 'stock_name'
})
output_df.to_pickle(f'../../Offline/backtest/backtest_data/test/{benchmark}_{test_start_date}_回归任务_v6.pkl') 

开始保存回测预测结果...
61/61 [==============================] - 7s 108ms/step


In [19]:
# plot_series_dist(output_df['label_pred'])

## 保存模型、预处理等模块

In [20]:
def save_json_file(data, path):
    import json
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def save_sklearn_pipeline(pipeline, path):
    import joblib
    joblib.dump(pipeline, path)

# 保存特征json
save_json_file(feature_config, '../../Online/enhance_sz50/feature_config.json')
# 保存特征预处理pipeline
save_sklearn_pipeline(feature_preprocess_pipeline, '../../Online/enhance_sz50/feature_preprocess_pipeline.joblib')
# 保存模型
model.save(f'../../Online/enhance_sz50/tf_models/')


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(227, 75), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x14df70ad0160>, 22949907461008), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(227, 75), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x14df70ad0160>, 22949907461008), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(75, 227), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x14df70c3c7f0>, 22949907460816), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(75, 227), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x14df70c3c7f0>, 22949907460816), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(227, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x14df70c403a0>, 22949910358976), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32, 1), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x14df71660460>, 22949905526096), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x14df7117a9d0>, 22949905526416), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x14df7117a9d0>, 22949905526416), {}).


INFO:tensorflow:Assets written to: ../../Online/enhance_sz50/tf_models/assets


INFO:tensorflow:Assets written to: ../../Online/enhance_sz50/tf_models/assets
